# Ranch Demo Notebook

This notebook has examples of how to make Ranch calls to run through the complete network building pipeline to develop standard roadway and transit network from scratch

In [12]:
import json
import os
from os.path import isdir
import pandas as pd
import geopandas as gpd
from pyproj import CRS

from osmnx.geocoder import geocode_to_gdf

import ranch
from ranch import sharedstreets
from ranch import Roadway
from ranch import Transit
from ranch import Parameters
from ranch.utils import link_df_to_geojson, point_df_to_geojson
from ranch.logger import RanchLogger

import sharedstreets.dataframe

In [ ]:
%load_ext autoreload
%autoreload 2

# Remote I/O

User to update, does not necessarily be the ranch directory

In [2]:
# project directory
root_dir = os.path.join("c:/apps/cowboy/sanjoaquin")

external_dir = os.path.join(root_dir, "data", "external")
interim_dir = os.path.join(root_dir, "data", "interim")

# software directory
ranch_dir = os.path.join("c:/apps/cowboy/Ranch")

# the folder where SharedStreets extractions live
shst_extract_dir = os.path.join(external_dir, "sharedstreets_extract")
osm_extract_dir = os.path.join(external_dir, "osmnx_extract")

for d in [root_dir, external_dir, interim_dir, ranch_dir, shst_extract_dir, osm_extract_dir]:
    if isdir(d):
        print(f"{d} exists!")
        continue
    
    print(f"creating {d}")
    os.makedirs(d)

c:/apps/cowboy/sanjoaquin exists!
c:/apps/cowboy/sanjoaquin\data\external exists!
c:/apps/cowboy/sanjoaquin\data\interim exists!
c:/apps/cowboy/Ranch exists!
c:/apps/cowboy/sanjoaquin\data\external\sharedstreets_extract exists!
c:/apps/cowboy/sanjoaquin\data\external\osmnx_extract exists!


In [3]:
parameters = Parameters(ranch_base_dir = ranch_dir)

2022-01-03 11:38:30, INFO: Lasso base directory set as: c:/apps/cowboy/Ranch


### Use the OSMNX Functionality to Grab a Geography
In this scenario, we are grabbing Lodi in California.

In [4]:
lodi = geocode_to_gdf("Lodi, California, USA")
#lodi.explore()

## Step 1 - SharedStreets Extraction

Extracts sharedstreets representation of OSM links.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from SharedStreets API.

In [6]:
shst = ranch.run_shst_extraction(lodi, output_dir=shst_extract_dir, pylib=True)
shst.explore()

## Step 2 - OSMNX Extraction

Extracts complete OSM attributes using OSMNX.
The input for this step is the polygon boundary file for the area.
The output for this step is geojson files from OSMNX.

In [7]:
# the OSMNX extraction file is saved to the 'output_dir' argument
osm_links, osm_nodes = ranch.run_osmnx_extraction(
    input_polygon_file = lodi,
    output_dir = None
)

## Step 3 - Consolidate SharedStreets and OSMNX exractions

consolidates raw roadway data and create initial roadway networks

In [9]:
# build and returns roaday network object from extraction files
roadway_network = Roadway.create_roadway_network_from_extracts(
    shst.reset_index(),
    osm_links,
    osm_nodes,
    parameters
)

2022-01-03 11:39:37, INFO: Removing duplicates in shst extraction data
2022-01-03 11:39:37, INFO: ...before removing duplicates, shst extraction has 2803 geometries.
2022-01-03 11:39:37, INFO: ...after removing duplicates, shst extraction has 2803 geometries.
2022-01-03 11:39:37, INFO: Extracting corresponding osm ways for every shst geometry
2022-01-03 11:39:40, INFO: shst extraction has 2803 geometries
2022-01-03 11:39:40, INFO: shst extraction has 2926 osm links
2022-01-03 11:39:40, INFO: ---joining osm shst with osmnx data---
2022-01-03 11:39:40, INFO: shst extraction has 0 two-way osm links
2022-01-03 11:39:40, INFO: and they are 0 geometrys
2022-01-03 11:39:40, INFO: after join, ther are 2926 osm links from shst extraction,     out of which there are 197 links that do not have osm info,     due to shst extraction (default tile 181224) contains 114 osm ids that are not included in latest OSM extraction,     e.g. private streets, closed streets.
2022-01-03 11:39:40, INFO: after joi

C:\Anaconda3\envs\ranch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Anaconda3\envs\ranch\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


2022-01-03 11:39:41, INFO: Lasso base directory set as: c:/apps/cowboy/Ranch


C:\Anaconda3\envs\ranch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Anaconda3\envs\ranch\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [10]:
RanchLogger.info("Initial network has {} links".format(roadway_network.links_df.shape[0]))
RanchLogger.info("Initial network has {} nodes".format(roadway_network.nodes_df.shape[0]))
RanchLogger.info("Initial network has {} shapes".format(roadway_network.shapes_df.shape[0]))

2022-01-03 11:39:49, INFO: Initial network has 2803 links
2022-01-03 11:39:49, INFO: Initial network has 1944 nodes
2022-01-03 11:39:49, INFO: Initial network has 2803 shapes


### Optional: If user wants to write out the roadway network in standard format, they can do the following, otherwise no need to write out

In [13]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step3_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step3_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(root_dir,"data", "interim","step3_links.geojson"), "w") as f:
    json.dump(link_geojson, f)


2022-01-03 11:41:58, INFO: write out shape geojson
2022-01-03 11:41:59, INFO: write out node geojson
2022-01-03 11:41:59, INFO: write out link geojson


## Step 4 - Third-Party Roadway Data Conflation

Conflates e.g. county network with roadway network using SharedStreets.
The input for this step is the 3rd-party geodatabases to conflate.
The output of this step is the SharedStreets conflation outputs in geojson.

In [14]:
# define the input 3rd-party file, either shapefile or geojson

input_network_file = os.path.join(external_dir, "sjmodel", "Network", "2015","TCM_MASTER_2A22_102717.shp")

In [15]:
# if the CRS is undefined in the input network file, user needs to specify:

input_crs = CRS("ESRI:102643")

In [ ]:
# the output is saved to the 'output_dir' argument
# user can specify the match option (reference here) 
# by 'custom_match_option'
# if not specified, use the default
# match option would vary by the input network

ranch.run_shst_match(
    input_network_file = input_network_file,
    input_crs = input_crs,
    output_dir = os.path.join(external_dir, "sjmodel", "shst_match"),
    custom_match_option = '--tile-hierarchy=8 --search-radius=50 --snap-intersections'
)

## Step 5 - Tidy Up Roadway

Identifies drive dead-ends, cul-de-secs, numbering links and nodes, etc.
The input for this step is the polygon file with subregion identifier, e.g. county.
This step labels each link and node with the county name, and assign model IDs for links and nodes.

In [ ]:
# this is the input county polygon file with county names

county_boundary_file = os.path.join(
    external_dir,
    "cb_2018_us_county_500k", 
    "san_joaquin.shp"
)

# specify the column to look for county name

county_variable_name = 'NAME'

In [ ]:
roadway_network.tidy_roadway(
    county_boundary_file = county_boundary_file,
    county_variable_name = county_variable_name
)

## Step 6 - Build Transit Network from GTFS

Build standard transit network from GTFS

In [ ]:
gtfs_dir = os.path.join(external_dir, "gtfs", "2015")

In [ ]:
# read gtfs into transit object

transit_network = Transit.load_all_gtfs_feeds(
    path = gtfs_dir,
    roadway_network= roadway_network,
    parameters=parameters
)

In [ ]:
# main activities - routing buses, creating rails

transit_network.build_standard_transit_network()

In [ ]:
# write out

transit_network.write_standard_transit(
    path = interim_dir
)

## Step 7 - Build Centroid Connectors

builds centroid connectors from TAZ, MAZ

In [ ]:
taz_polygon_file = os.path.join(
    external_dir,
    "taz",
    "SJ_TAZ_Aug2011.shp"
)

In [ ]:
roadway_network.build_centroid_connectors(
    build_taz_drive = True,
    build_taz_active_modes = True,
    input_taz_polygon_file = taz_polygon_file
)

## Step 8 - Write out Standard Format

In [ ]:
roadway_network.standard_format(
    county_boundary_file = os.path.join(root_dir,"data", "external","cb_2018_us_county_500k", "san_joaquin.shp"),
    county_variable_name = 'NAME'
)

In [ ]:
RanchLogger.info("write out shape geojson")

shape_prop = ['id', 'fromIntersectionId', 'toIntersectionId', 'forwardReferenceId', 'backReferenceId']
shape_geojson = link_df_to_geojson(
    roadway_network.shapes_df, 
    shape_prop
)

with open(os.path.join(interim_dir,"step8_shapes.geojson"), "w") as f:
    json.dump(shape_geojson, f)

RanchLogger.info("write out node geojson")

node_prop = roadway_network.nodes_df.drop('geometry', axis = 1).columns.tolist()
node_geojson = point_df_to_geojson(
    roadway_network.nodes_df, 
    node_prop
)

with open(os.path.join(interim_dir,"step8_nodes.geojson"), "w") as f:
    json.dump(node_geojson, f)

RanchLogger.info("write out link geojson")

link_prop = roadway_network.links_df.drop('geometry', axis = 1).columns.tolist()
link_geojson = link_df_to_geojson(
    roadway_network.links_df, 
    link_prop
)

with open(os.path.join(interim_dir,"step8_links.geojson"), "w") as f:
    json.dump(link_geojson, f)
